## Foundry 에이전트에서 MCP 도구 사용하기

## 🚧 **공사 중 (Under Construction)** 🚧

> 이 문서는 현재 업데이트 작업 중입니다. 실습을 권장하지 않습니다.  
> 아래의 내용은 현재 올바로 동작하지 않으며 언제든 변경될 수 있습니다.  
> 정비가 완료되면 이 안내는 사라집니다.  
> 불편을 드려 죄송합니다. 🙏

원격 Model Context Protocol (MCP) 서버에 호스팅되는 도구에 연결하여 Foundry 에이전트의 기능을 확장할 수 있습니다.

Key Features

Hosted MCP Server: The MCP server is hosted and managed by Foundry, eliminating the need to manage server infrastructure
Persistent Agents: Agents are created and stored server-side, allowing for stateful conversations
Tool Approval Workflow: Configurable approval mechanisms for MCP tool invocations


In [2]:
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential
from agent_framework import Agent

"""Basic example of Foundry agent with hosted MCP tools."""

client=AzureAIAgentClient(credential=DefaultAzureCredential())

# Create a hosted MCP tool using the client method
learn_mcp = client.get_mcp_tool(
    name="Microsoft Learn MCP",
    url="https://learn.microsoft.com/api/mcp",
    approval_mode="never_require",  # 승인 없이 MCP 도구 자동 실행
)

# Create agent with hosted MCP tool
agent = client.as_agent(
    name="MicrosoftLearnAgent", 
    instructions="You answer questions by searching Microsoft Learn content only.",
    tools=learn_mcp,
)

# Simple query without approval workflow
result = await agent.run(
    "Please summarize the Azure AI Agent documentation related to MCP tool calling?"
)
print(result)


Can't parse tool.


Here’s a summary of Azure AI Agent documentation related to MCP (Model Context Protocol) tool calling:

---

## Overview

Azure AI Agents can call and integrate with external tools using the Model Context Protocol (MCP), enabling them to dynamically augment their capabilities (e.g., database queries, calling APIs) at runtime. The MCP tool ecosystem is used to expose, discover, and invoke these external tools securely.

---

## Key Concepts

- **MCP Tools**: Functions or APIs exposed via an MCP server, which agents can discover and invoke as tools during their runtime.
- **MCP Server/Client**: The MCP server hosts and exposes tools, while the MCP client (within the AI agent environment) discovers and invokes these tools.
- **Authentication**: Connections to MCP tools can be unauthenticated or authenticated using Microsoft Entra (formerly Azure Active Directory) with options like project managed identity.
- **Use Cases**: Agents can perform database queries, trigger external APIs, or exe